In [5]:
size = 256
keys = [(i, i) for i in range(size)]

In [6]:
lookup_dict = {k: k for k in keys}

In [7]:
from functools import cache


@cache
def lookup_cache(key):
    return lookup_dict.get(key)

In [8]:
import numpy as np

lookup_array = np.zeros((size, size), dtype="object")
for k in keys:
    lookup_array[k] = k

In [9]:
%%timeit
lookup_dict.get((100, 100))

26.6 ns ± 0.697 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
%%timeit
lookup_cache((100, 100))

41.4 ns ± 0.962 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [11]:
%%timeit
lookup_array[100, 100]

33.9 ns ± 0.73 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
